In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import datasets
from sklearn import metrics
from sklearn.decomposition import PCA

In [2]:
housing = pd.DataFrame(pd.read_csv('./Housing.csv'))
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
# Any dataset that has columns with values as 'Yes' or 'No', strings' values cannot be used.
# However, we can convert them to numerical values as binary.

varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

housing[varlist] = housing[varlist].apply(binary_map)

housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [4]:
X = housing.values[:,1:12]
print('X =', X[0:5])
Y = housing.values[:,0]
print('Y =', Y[0:5])

X = [[7420 4 2 3 1 0 0 0 1 2 1]
 [8960 4 4 4 1 0 0 0 1 3 0]
 [9960 3 2 2 1 0 1 0 0 2 1]
 [7500 4 2 2 1 0 1 0 1 3 1]
 [7420 4 1 2 1 1 1 0 1 2 0]]
Y = [13300000 12250000 12250000 12215000 11410000]


In [5]:
# Implementation SVR Regression (Linear Kernel) over the numbers of principal components from PCA extraction scaling. 

Prin_Comp = []
Accuracy_lin = []
max_accuracy_lin = 0 
max_comp_lin = 0

N_Comp = range(1,11)
for N in N_Comp:
    Prin_Comp.append(N)
    pca = PCA(n_components = N)
    principalComponents = pca.fit_transform(X)
    principalDF = pd.DataFrame(data = principalComponents)
    X_train, X_test, Y_train, Y_test = train_test_split(principalDF, Y, test_size=0.2, random_state = 0)
    sc_X = StandardScaler()
    X_train_sc = sc_X.fit_transform(X_train)
    X_test_sc = sc_X.fit_transform(X_test)
    model_lin = SVR(kernel = 'linear', C = 1e6)
    model_lin.fit(X_train_sc,Y_train)
    Y_pred_lin = model_lin.predict(X_test_sc)
    Accuracy_lin.append(model_lin.score(X_test_sc, Y_test))
    if model_lin.score(X_test_sc, Y_test) > max_accuracy_lin:
        max_comp_lin = N
        max_accuracy_lin = model_lin.score(X_test_sc, Y_test)
print("The N of principal components with the highest accuracy is:", max_comp_lin, "components with the accuracy of", max_accuracy_lin)

The N of principal components with the highest accuracy is: 8 components with the accuracy of 0.688192071784399


In [6]:
# Implementation SVR Regression (Polynomial Kernel) over the numbers of principal components from PCA extraction scaling. 

Prin_Comp = []
Accuracy_poly = []
max_accuracy_poly = 0 
max_comp_poly = 0

N_Comp = range(1,11)
for N in N_Comp:
    Prin_Comp.append(N)
    pca = PCA(n_components = N)
    principalComponents = pca.fit_transform(X)
    principalDF = pd.DataFrame(data = principalComponents)
    X_train, X_test, Y_train, Y_test = train_test_split(principalDF, Y, test_size=0.2, random_state = 0)
    sc_X = StandardScaler()
    X_train_sc = sc_X.fit_transform(X_train)
    X_test_sc = sc_X.fit_transform(X_test)
    model_poly = SVR(kernel = 'poly', C = 1e5)
    model_poly.fit(X_train_sc,Y_train)
    Y_pred_poly = model_poly.predict(X_test_sc)
    Accuracy_poly.append(model_poly.score(X_test_sc, Y_test))
    if model_poly.score(X_test_sc, Y_test) > max_accuracy_poly:
        max_comp_poly = N
        max_accuracy_poly = model_poly.score(X_test_sc, Y_test)
print("The N of principal components with the highest accuracy is:", max_comp_poly, "components with the accuracy of", max_accuracy_poly)

The N of principal components with the highest accuracy is: 5 components with the accuracy of 0.5354556738266577


In [7]:
# Implementation SVR Regression (Radial Basis Function Kernel) over the numbers of principal components from PCA extraction scaling. 

Prin_Comp = []
Accuracy_rbf = []
max_accuracy_rbf = 0 
max_comp_rbf = 0

N_Comp = range(1,11)
for N in N_Comp:
    Prin_Comp.append(N)
    pca = PCA(n_components = N)
    principalComponents = pca.fit_transform(X)
    principalDF = pd.DataFrame(data = principalComponents)
    X_train, X_test, Y_train, Y_test = train_test_split(principalDF, Y, test_size=0.2, random_state = 0)
    sc_X = StandardScaler()
    X_train_sc = sc_X.fit_transform(X_train)
    X_test_sc = sc_X.fit_transform(X_test)
    model_rbf = SVR(kernel = 'rbf', C = 1e6)
    model_rbf.fit(X_train_sc,Y_train)
    Y_pred_rbf = model_rbf.predict(X_test_sc)
    Accuracy_rbf.append(model_rbf.score(X_test_sc, Y_test))
    if model_rbf.score(X_test_sc, Y_test) > max_accuracy_rbf:
        max_comp_rbf = N
        max_accuracy_rbf = model_rbf.score(X_test_sc, Y_test)
print("The N of principal components with the highest accuracy is:", max_comp_rbf, "components with the accuracy of", max_accuracy_rbf)

The N of principal components with the highest accuracy is: 10 components with the accuracy of 0.5834690324496695


In [8]:
print(Prin_Comp)
print(Accuracy_lin)
print(Accuracy_poly)
print(Accuracy_rbf)
plt.plot(Prin_Comp, Accuracy_lin, label = 'Linear Model')
plt.plot(Prin_Comp, Accuracy_poly, label = 'Polynomial Model')
plt.plot(Prin_Comp, Accuracy_rbf, label = 'Radial Basis Function Model')
plt

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[0.30235940093081093, 0.5528365189443171, 0.576537771042675, 0.5715492344458598, 0.6430060279187617, 0.6687653253148902, 0.6675584538601521, 0.688192071784399, 0.6840595209151683, 0.6855437496207218]
[0.01192617407138652, 0.16281298061682992, 0.12669699304906767, 0.4261560641151143, 0.5354556738266577, 0.33211271412637167, 0.4144159503771542, 0.5088730505673984, 0.5306905176617527, 0.5095918805456405]
[0.29757056925073266, 0.5011501990687838, 0.4848868370722845, 0.48316431320220277, 0.5596146066322106, 0.5511067830963237, 0.5467897226142022, 0.5630940573039944, 0.5698116109808598, 0.5834690324496695]
